In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import kinodyn_um.urdfparser as u2c
import numpy as np
import casadi as ca

import matplotlib.pyplot as plt

In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# 1. Load robot model
# ─────────────────────────────────────────────────────────────────────────────
alpha = u2c.URDFparser()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(project_root, 'usage', 'urdf', 'alpha_5_robot.urdf')
alpha.from_file(path_to_urdf)

root = "base_link"
tip  = "alpha_standard_jaws_base_link"
n_joints = alpha.get_n_joints(root, tip)

F_next = alpha.forward_simulation(root, tip)

In [ ]:
# fixed sizes -------------------------------------------------------------
n_theta   = 32
nx        = 2*n_joints                      # 8   (q ‖ q̇)
nv        = n_joints                        # 4   velocity rows
lam       = 1e-4                            # Tikhonov damping
N         = 50                              # prediction horizon (set as you like)

# ------------------------------------------------------------------------
# 1)  STEP function: one forward‑dynamics step
# ------------------------------------------------------------------------
x_prev   = ca.SX.sym("x_prev",  nx)
tau_k    = ca.SX.sym("tau_k",   n_joints)

# constant‑within‑horizon symbols
dt        = ca.SX.sym("dt")
g         = ca.SX.sym("g")
payload   = ca.SX.sym("payload_props", 4)
theta     = ca.SX.sym("theta",  n_theta)
low_lim   = ca.SX.sym("low_lim", n_joints)
up_lim    = ca.SX.sym("up_lim", n_joints)
eps_T     = ca.SX.sym("eps_T",  n_joints)

x_next = F_next(
    x_prev, tau_k,
    dt, g, payload,
    theta, low_lim, up_lim, eps_T
)

step = ca.Function(
    "step",
    [x_prev, tau_k, dt, g, payload, theta, low_lim, up_lim, eps_T],
    [x_next]
)

# ------------------------------------------------------------------------
# 2)  ROLLOUT with mapaccum  (serial scan)
# ------------------------------------------------------------------------
rollout = step.mapaccum(N)   # returns stack of N states

#  inputs to the rollout --------------------------------------------------
x0          = ca.SX.sym("x0", nx)               # single initial state
tau_seq     = ca.SX.sym("tau_seq", n_joints, N) # τ₀ … τ_{N‑1}
dt_seq      = ca.SX.sym("dt", 1, N)
g_sym       = ca.SX.sym("g")
payload_sym = ca.SX.sym("payload_props", 4)
theta_prev  = ca.SX.sym("theta_prev", n_theta)
low_lim_sym = ca.SX.sym("low_lim",  n_joints)
up_lim_sym  = ca.SX.sym("up_lim",   n_joints)
epsT_sym    = ca.SX.sym("eps_T",    n_joints)

X_pred = rollout(
    x0,          # initial state
    tau_seq,     # varying input  (n_joints × N)
    dt_seq, g_sym, payload_sym,
    theta_prev, low_lim_sym, up_lim_sym, epsT_sym
)                       # shape:  nx × N   (states x₁ … x_N)

# ------------------------------------------------------------------------
# 3)  build residual stack (velocity only) and Jacobian wrt θ
# ------------------------------------------------------------------------
v_meas = ca.SX.sym("x_meas_tail", 4, N)    # measured x₁ … x_N
v_pred  = X_pred[4:8, :]                         # rows 4‑7  -> nv × N

E = ca.reshape(v_meas - v_pred, nv*N, 1)         # (N*nv) × 1 residual vector
J = ca.jacobian(E, theta_prev)                   # (N*nv) × 32   regressor

# damped Gauss–Newton step ----------------------------------------------
JTJ  = ca.mtimes(J.T, J) + lam*ca.SX.eye(n_theta)
gain = ca.solve(JTJ, J.T)                        # 32 × (N*nv)
dtheta = ca.mtimes(gain, E)
theta_next = theta_prev + dtheta

# ------------------------------------------------------------------------
# 4)  wrap everything into one callable function
# ------------------------------------------------------------------------
update_theta_mapaccum = ca.Function(
    "update_theta_mapaccum",
    [theta_prev, x0, v_meas, tau_seq,
     dt_seq, g_sym, payload_sym, low_lim_sym, up_lim_sym, epsT_sym],
    [theta_next, E, dtheta, v_pred],
)

In [ ]:
# F_next.save('arm.casadi')

# c , cpp or matlab code generation for forward dynamics
F_next.generate("F_next_.c")
os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

print('✔ arm.casadi saved — ready for simulation')